<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.3: Control Flow
**Prev: [Combinational Logic](2.2_comb_logic.ipynb)**<br>
**Next: [Sequential Logic](2.4_sequential_logic.ipynb)**

## Motivation

지금까지 Chisel에서는 소프트웨어와 하드웨어 사이에 강력한 대응 관계가 있었습니다.
control flow에서는 우리가 둘에 대해 생각하는 방식 사이에 더 큰 차이가 있을 것입니다.
이 모듈은 생성기 소프트웨어와 하드웨어 모두에서 control flow을 소개합니다.
Chisel 와이어에 다시 연결하면 어떻게 됩니까?
두 개 이상의 입력으로 mux를 어떻게 만들 수 있습니까?
이 모듈을 완료하면 이러한 질문 등에 대한 답을 얻을 수 있습니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Last Connect Semantics

<span style="color:blue">**Example: Reassignment**</span><br>
앞에서 보았듯이 Chisel을 사용하면 `:=` 연산자를 사용하여 구성 요소를 연결할 수 있습니다.
여러 가지 이유로 동일한 구성 요소에 여러 connect 구문을 실행할 수 있습니다.
이 경우 마지막 구문이 win합니다.

In [3]:
class LastConnect extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := 1.U
  io.out := 2.U
  io.out := 3.U
  io.out := 4.U
}

//  Test LastConnect
test(new LastConnect) { c => c.io.out.expect(4.U) } // Assert that the output correctly has 4
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
test LastConnect Success: 0 tests passed in 2 cycles in 0.016582 seconds 120.61 Hz
SUCCESS!!


defined class LastConnect

---
# `when`, `elsewhen`, and `otherwise`
조건부 논리의 Chisel의 기본 구현은 `when`, `elsewhen` 및 `otherwise` 구문입니다.
이것은 일반적으로 다음과 같습니다
```scala
when(someBooleanCondition) {
  // things to do when true
}.elsewhen(someOtherBooleanCondition) {
  // things to do on this condition
}.otherwise {
  // things to do if none of th boolean conditions are true
}
```

위의 순서대로 나타나야 하지만 후자 중 하나를 생략할 수 있습니다.
원하는 만큼 elsewhen 절이 있을 수 있습니다.
true인 섹션은 구성을 종료합니다.
세 body에서 취한 행동은 복잡한 블록이 될 수 있으며 중첩된 `when`와 allies을 포함할 수 있습니다.
Scala `if`와 **달리** `when`과 관련된 블록은 값을 반환하지 않습니다.
말할 수 없다

```scala
val result = when(squareIt) { x * x }.otherwise { x }
```
본 구문은 작동하지 **않습니다**. 이에 대한 해결책은 *Wires* 섹션에서 논의할 것입니다.

<span style="color:blue">**Example: Chisel Conditionals**</span><br>
다음은 `when` 구문을 사용하는 `Module`의 예입니다.

In [4]:
// Max3 returns the max of its 3 arguments
class Max3 extends Module {
  val io = IO(new Bundle {
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out = Output(UInt(16.W))
  })
    
  when(io.in1 >= io.in2 && io.in1 >= io.in3) {
    io.out := io.in1  
  }.elsewhen(io.in2 >= io.in3) {
    io.out := io.in2 
  }.otherwise {
    io.out := io.in3
  }
}

// Test Max3
test(new Max3) { c =>
  // verify that the max of the three inputs is correct
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(2.U)
  c.io.out.expect(6.U)  // input 1 should be biggest
  c.io.in2.poke(7.U)
  c.io.out.expect(7.U)  // now input 2 is
  c.io.in3.poke(11.U)
  c.io.out.expect(11.U) // and now input 3
  c.io.in3.poke(3.U)
  c.io.out.expect(7.U)  // show that decreasing an input works as well
  c.io.in1.poke(9.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(6.U)
  c.io.out.expect(9.U)  // still get max with tie
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
test Max3 Success: 0 tests passed in 2 cycles in 0.004943 seconds 404.58 Hz
SUCCESS!!


defined class Max3

---
# The `Wire` Construct

`when`가 값을 반환하지 않는다는 제한 사항을 설명하는 위의 설명으로 돌아가 보겠습니다.
Chisel `Wire` 구조는 이 문제를 해결하는 방법 중 하나입니다.
`Wire`는 연결 `:=` 연산자의 우항 또는 좌항에 나타날 수 있는 회로 구성요소를 정의합니다.

<span style="color:blue">**Example: 4-Input Sort with Wires**</span><br>
이를 설명하기 위해 네 개의 숫자 입력을 다음과 같이 정렬하는 작은 조합 정렬기를 만들어 보겠습니다.
네 개의 숫자 출력. 더 명확하게 하기 위해 다음 그래프를 고려하십시오. 데이터는 빨간색 선을 따릅니다.
왼쪽 값이 오른쪽보다 작을 때 각 단계에서 왼쪽이 오른쪽보다 클 때 값을 교환하는 검은색 선을 따릅니다.
![Sort4](images/Sorter4.png)
다이어그램은 이름이 *row*로 시작하는 일련의 셀을 보여줍니다. 우리는 `Wire`를 사용하여 특정 복사 또는 스왑의 결과가 배치되는 위치로 구성할 것입니다. 이에 대한 코드는 매우 장황하지만 나중에 축소하는 방법을 보게 될 것입니다.

In [6]:
/** Sort4 sorts its 4 inputs to its 4 outputs */
class Sort4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(io.in0 < io.in1) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(io.in2 < io.in3) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(row11 < row12) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  val row20 = Wire(UInt(16.W))
  val row23 = Wire(UInt(16.W))
  when(row10 < row13) {
    row20 := row10            // preserve middle 2 elements
    row23 := row13
  }.otherwise {
    row20 := row13            // swap middle two elements
    row23 := row10
  }

  when(row20 < row21) {
    io.out0 := row20            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row20
  }

  when(row22 < row23) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row23
  }.otherwise {
    io.out2 := row23            // swap first two elements
    io.out3 := row22
  }
}


// Here's the tester
test(new Sort4) { c =>
  // verify the inputs are sorted
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(3.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(9.U)
  c.io.out3.expect(12.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


Elaborating design...
Done elaborating.
test Sort4 Success: 0 tests passed in 2 cycles in 0.003785 seconds 528.45 Hz
SUCCESS!!


defined class Sort4

다음은 Scala `List` 기능을 사용하는 보다 철저한 테스터입니다. 이후 모듈에서 더 많은 `List` 기능을 볼 수 있습니다.

In [7]:
// Here's the tester
test(new Sort4) { c =>
  // verify the all possible ordering of 4 numbers are sorted
  List(1, 2, 3, 4).permutations.foreach { case i0 :: i1 :: i2 :: i3 :: Nil =>
    println(s"Sorting $i0 $i1 $i2 $i3")
    c.io.in0.poke(i0.U)
    c.io.in1.poke(i1.U)
    c.io.in2.poke(i2.U)
    c.io.in3.poke(i3.U)
    c.io.out0.expect(1.U)
    c.io.out1.expect(2.U)
    c.io.out2.expect(3.U)
    c.io.out3.expect(4.U)
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
Sorting 1 2 3 4
Sorting 1 2 4 3
Sorting 1 3 2 4
Sorting 1 3 4 2
Sorting 1 4 2 3
Sorting 1 4 3 2
Sorting 2 1 3 4
Sorting 2 1 4 3
Sorting 2 3 1 4
Sorting 2 3 4 1
Sorting 2 4 1 3
Sorting 2 4 3 1
Sorting 3 1 2 4
Sorting 3 1 4 2
Sorting 3 2 1 4
Sorting 3 2 4 1
Sorting 3 4 1 2
Sorting 3 4 2 1
Sorting 4 1 2 3
Sorting 4 1 3 2
Sorting 4 2 1 3
Sorting 4 2 3 1
Sorting 4 3 1 2
Sorting 4 3 2 1
test Sort4 Success: 0 tests passed in 2 cycles in 0.018270 seconds 109.47 Hz
SUCCESS!!


---
# Exercises

<span style="color:red">**Exercise: Polynomial Circuit**</span><br>
이 다항식의 값을 계산할 `Module`을 빌드하십시오.
- $x^2 - 2x + 1$
- $2x^2 + 6x + 3$
- $4x^2 - 10x -5$

선택기 입력은 계산할 다항식을 결정합니다. $x^2$ 계산이 모듈에서 한 번만 나타나고 출력에 대한 단일 연결이 있도록 `Wire`를 사용합니다.

먼저 테스트 주도 개발을 사용하고 스칼라를 사용하여 모델을 작성해 보겠습니다. 아래 어설션을 전달하려면 이 함수 정의를 완료하세요. 철저한(exhaustive) 점검이 아니라 온전한(sanity) 점검입니다.

In [ ]:
def poly0(x: Int): Int = ???
def poly1(x: Int): Int = ???
def poly2(x: Int): Int = ???

assert(poly0(0) == 1)
assert(poly1(0) == 3)
assert(poly2(0) == -5)

assert(poly0(1) == 0)
assert(poly1(1) == 11)
assert(poly2(1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly0(x: Int): Int = x*x - 2*x + 1
def poly1(x: Int): Int = 2*x*x + 6*x + 3
def poly2(x: Int): Int = 4*x*x - 10*x - 5
</pre></article></div></section></div>

더 쉽게 만들기 위해 원하는 하드웨어 모듈처럼 작동하는 함수를 만들어 보겠습니다. Scala `if` 문을 사용하여 `select` 입력을 기반으로 다항식을 선택합니다.

In [8]:
def poly(select: Int, x: Int): Int = {
  ???
}

assert(poly(1, 0) == 3)
assert(poly(1, 1) == 11)
assert(poly(2, 1) == -11)

: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly(select: Int, x: Int): Int = {
  if(select == 0) {
    poly0(x)
  }
  else if(select == 1) {
    poly1(x)
  }
  else {
    poly2(x)
  }
}
</pre></article></div></section></div>

값이 맞는 것 같습니다. 이제 다음 템플릿을 사용하여 회로를 구현하십시오.

In [ ]:
// compute the polynomial
class Polynomial extends Module {
  val io = IO(new Bundle {
    val select = Input(UInt(2.W))
    val x = Input(SInt(32.W))
    val fOfX = Output(SInt(32.W))
  })
    
  val result = Wire(SInt(32.W))  
  val square = Wire(SInt(32.W))  
  
  ???

  io.fOfX := result  
}

// Test Polynomial
test(new Polynomial) { c =>
  for(x <- 0 to 20) {
    for(select <- 0 to 2) {
      c.io.select.poke(select.U)
      c.io.x.poke(x.S)
      c.io.fOfX.expect(poly(select, x).S)
    }
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  square := io.x * io.x
  when(io.select === 0.U) {
    result := (square - (2.S * io.x)) + 1.S
  }.elsewhen(io.select === 1.U) {
    result := (2.S * square) + (6.S * io.x) + 3.S
  }.otherwise {
    result := (4.S * square) - (10.S * io.x) - 5.S
  }
</pre></article></div></section></div>

<span style="color:red">**Exercise: Finite State Machine**</span><br>
Karnaugh 맵을 사용하여 상태 머신의 논리를 최적화하는 것은 지루하고 합성 도구로 해결됩니다. 또한 직관적이지 않고 읽을 수 없는 코드를 생성합니다. 따라서 Chisel control flow과 last connect semantics를 사용하여 보다 합리적인 방법으로 작성할 것입니다.

대학원생은 유휴(Idle), 코딩(Coding), 작문(Writing), 졸업(Graduating)의 4개 상태를 거치게 됩니다. 이러한 상태는 세 가지 입력(커피, 그들이 생각해낸 아이디어, 진행 상황에 대한 조언자의 압력)을 기반으로 전환됩니다. 졸업하면 Idle 상태로 돌아갑니다. 아래 FSM 다이어그램은 이러한 상태와 전환을 보여줍니다. 레이블이 지정되지 않은 전환(즉, 입력이 없는 경우)은 대학원생을 현재 상태에 머무르는 대신 Idle 상태로 되돌립니다. 입력 우선순위는 커피 > 아이디어 > 압력이므로 Idle 상태에서 커피와 압력을 동시에 받으면 대학원생은 Coding 상태로 이동합니다.

<img src="images/fsm.png" width="500" />

먼저 하드웨어에 대해 테스트할 모델을 구성합니다. 상태 머신에 대한 다음 기능 설명을 완성하십시오. 4개의 입력이 있습니다. 출력은 다음 상태입니다. 상태 지도가 제공됩니다. `states("grad")`와 같이 액세스할 수 있습니다.

In [ ]:
// state map
def states = Map("idle" -> 0, "coding" -> 1, "writing" -> 2, "grad" -> 3)

// life is full of question marks
def gradLife (state: Int, coffee: Boolean, idea: Boolean, pressure: Boolean): Int = {
  var nextState = states("idle")
  ???
  nextState
}

// some sanity checks
(0 until states.size).foreach{ state => assert(gradLife(state, false, false, false) == states("idle")) }
assert(gradLife(states("writing"), true, false, true) == states("writing"))
assert(gradLife(states("idle"), true, true, true) == states("coding"))
assert(gradLife(states("idle"), false, true, true) == states("idle"))
assert(gradLife(states("grad"), false, false, false) == states("idle"))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  if (state == states("idle")) {
    if      (coffee) { nextState = states("coding") }
    else if (idea) { nextState = states("idle") }
    else if (pressure) { nextState = states("writing") }
  } else if (state == states("coding")) {
    if      (coffee) { nextState = states("coding") } 
    else if (idea || pressure) { nextState = states("writing") }
  } else if (state == states("writing")) {
    if      (coffee || idea) { nextState = states("writing") }
    else if (pressure) { nextState = states("grad") }
  }
</pre></article></div></section></div>

아직 순차 논리를 배우지 않았기 때문에 현재 상태는 `Module`에 대한 입력이고 다음 상태는 이전의 `gradLife` 함수와 마찬가지로 출력입니다. 이제 테스터를 통과하기 위해 Chisel에서 상태 머신을 구현합니다. Chisel은 `Enum`이라는 편리한 상태 머신 매핑 기능을 제공합니다. 이러한 상태를 사용하려면 `UInt` 리터럴처럼 취급하세요. 하드웨어 equality는 트리플 등호(===)로 수행된다는 것을 기억하십시오!

In [ ]:
// life gets hard-er
class GradLife extends Module {
  val io = IO(new Bundle {
    val state = Input(UInt(2.W))
    val coffee = Input(Bool())
    val idea = Input(Bool())
    val pressure = Input(Bool())
    val nextState = Output(UInt(2.W))
  })
    
  val idle :: coding :: writing :: grad :: Nil = Enum(4)
  
  ???
}


// Test
test(new GradLife) { c =>
  // verify that the hardware matches the golden model
  for (state <- 0 to 3) {
    for (coffee <- List(true, false)) {
      for (idea <- List(true, false)) {
        for (pressure <- List(true, false)) {
          c.io.state.poke(state.U)
          c.io.coffee.poke(coffee.B)
          c.io.idea.poke(idea.B)
          c.io.pressure.poke(pressure.B)
          c.io.nextState.expect(gradLife(state, coffee, idea, pressure).U)
        }
      }
    }
  }
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-5" />
<label for="check-5"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.nextState := idle
  when (io.state === idle) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea) { io.nextState := idle }
    .elsewhen (io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === coding) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea || io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === writing) {
    when      (io.coffee || io.idea) { io.nextState := writing }
    .elsewhen (io.pressure) { io.nextState := grad }
  }
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)